## Overview
Spring Retry allows to declaratively and programmatically retry method execution on failure. Starting with Spring 7, it is included in the framework. Prior to it, it is available as a separate dependency `org.springframework.retry:spring-retry`.

## Declarative Usage
Spring Retry allows us to annotate methods that should be retried using `@Retryable` annotation. However, to enable the annotation, the configuration must contain:

In [ ]:
@Configuration
@EnableRetry
public class AppConfiguration {
    // ...
}

Now we can annotate methods with `@Retryable` and Spring Retry will automatically reexecute the method:

In [ ]:
@Retryable()
public int getRandomEvenInteger() {
    // Generate random integer
    Random random = new Random();
    int number = random.nextInt();
    if (number % 2 != 0) {
        throw new OddRandomException();
    }

    return number;
}

The `@Retryable`annotation has several properties that controls how and when retries are attempted:
- `retryFor`: list of `Throwable` that would trigger retry. If the exeption thrown by method is outside of this list, no retry will be attempted. If this is not set then all exceptions are retried
- `noRetryFor`: list of `Throwable` for which retry should not be attempted.
- `maxAttempts`: maximum number of times retries should be done, defaults to 3
- `backoff`: specifies the backoff policy

### Backoff
Allows us to configure a delay between subsequent retries. As an example:

In [ ]:
@Retryable(
    backoff = @Backoff(
        delay = 500, // initial delay defaults to 1000 ms
        maxDelay = 5000, // maximum delay, defaults to 0 which means maxDelay can be anything
        multiplier = 2, // exponential backoff initialDelay × (multiplier)^(n − 1), default 1
        random = false, // randomise delay between initial delay and calculated delay
    )
)

// Use delayExpression, maxDelayExpression and multiplierExpression to set the value through SpEL
// multiplierExpression = "${retry.multiplier}"

With the above configuration, the delays are (as per $delay \times multiplier^{n-1}$):
```
1st attempt: 500
2nd attempt: 1000
3rd attempt: 2000
4th attempt: 4000
5th attempt: 5000
6th attempt: 5000
...
```

If random was set to true, it could be (as per $nextDelay = currrentDelay\times(1 + random \times (multiplier -1))$):
```
1st attempt: 500   // Initial delay
2nd attempt: 654   // between [0, 1000)
3rd attempt: 1920  // between [0, 2000)
4th attempt: 901   // between [0, 4000)
5th attempt: 3500  // between [0, 5000)
6th attempt: 2500  // between [0, 5000)
...
```

### Recovery
When all retries are exhausted, the last exception is thrown. We can annotate a method with `@Recover` to execute after last unsuccessful retry:

In [ ]:
@Recover
public int recover(OddRandomException ex) {
    return 2;
}

There can be multiple methods annotated with `@Recover`, Spring will chose the most appropriate one by:
- Checking for most matching Exception parameter

In [ ]:
@Recover
public int recover(OddRandomException ex) { // --> this one wins
    return 2;
}

@Recover
public int recover2(Exception ex) {
    return 3;
}

- Checking the recovery method return type:

In [ ]:
@Recover
public String recover(OddRandomException ex) {
    return "2";
}

@Recover
public int recover() { // --> this one wins
    return 2;
}

Basically choose the method that makes the most sense in the given context. You can also specify specific method using `recover` property of `@Retryable`.

## Programmatic Usage
Use `RetryTemplate` to trigger retries. The builder is most commonly used to initiate it:

In [ ]:
public int getRandomOddInteger() {
    RetryTemplate retryTemplate = RetryTemplate.builder()
            .maxAttempts(3)
            .exponentialBackoff(500, 2.0, 5000)
            .retryOn(EvenRandomException.class)
            .build();

    Random random = new Random();
    return retryTemplate.execute(context -> {
        // Generate random integer
        int number = random.nextInt();
        if (number % 2 == 0) {
            throw new EvenRandomException();
        }

        return number;
    });
}

`RetryTemplate` implements `RetryOperations` which contains different overloaded variants of the `execute` method:

In [ ]:
public interface RetryOperations {
	<T, E extends Throwable> T execute(RetryCallback<T, E> retryCallback) throws E;
	<T, E extends Throwable> T execute(RetryCallback<T, E> retryCallback, RecoveryCallback<T> recoveryCallback) throws E;
	<T, E extends Throwable> T execute(RetryCallback<T, E> retryCallback, RetryState retryState) throws E, ExhaustedRetryException;
    // ...
}

`RetryCallback` is a functional interface defining one method `T doWithRetry(RetryContext context) throws E`. Thus while executing anything inside a `RetryTemplate` we have access to `RetryContext` which contains information about the current retry count or last exception. Some implementation also contain a map where we can store data to persist between retries. In declarative approach, we get `RetryContext` using `RetrySynchronizationManager.getContext();`

In [ ]:
retryTemplate.execute(context -> {
    if (context.getRetryCount() > 0) {
        System.out.println("Current attempt count = " + context.getRetryCount());
        System.out.println("Last generated number was = " + context.getAttribute("lastNum"));
    }
    
    // Generate random integer
    int number = random.nextInt();
    context.setAttribute("lastNum", number);
    if (number % 2 == 0) {
        throw new EvenRandomException();
    }

    return number;
});

The builder contains a number of methods that can help us configure `RetryTemplate` in various ways:
<div style="display: inline-block" />

| Method                                   | Description                                                        |
|------------------------------------------|--------------------------------------------------------------------|
| `maxAttempts(int)`                       | Sets the total number of attempts                                  |
| `retryOn(Class<? extends Throwable>)`    | Exceptions to consider for retry                                   |
| `notRetryOn(Class<? extends Throwable>)` | Exceptions to not retry on                                         |
| `traversingCauses()`                     | Check exception's cause tree to find the retryOn exception         |
| `customRetryPolicy(RetryPolicy)`         | Allows to plug in a completely custom `RetryPolicy` implementation |
| `infiniteRetry()`                        | Use `AlwaysRetryPolicy`                                            |
| `withTimeout(long)`                      | Use `TimeoutRetryPolicy`                                           |
| `noBackoff()`                            | Immediately retries                                                |
| `fixedBackoff(long)`                     | Constant backoff                                                   |
| `exponentialBackoff(long, double, long)` | Exponential backoff given delay, multiplier and maxDelay           |
| `uniformRandomBackoff(long, long)`       | Random delay between the given range                               |
| `customBackoff(BackoffPolicy)`           | Allows to provide a custom `BackOffPolicy`                         |
</div>

### Retry Policy
`RetryPolicy` controls if retry attempt should be made and how many times. Some built-in implementations:
- `SimpleRetryPolicy`: allows a fixed number of attempts and can filter by exception types.

In [ ]:
// Retries exactly 3 times only for SQL Exceptions
SimpleRetryPolicy policy = new SimpleRetryPolicy(3, 
    Collections.singletonMap(SQLException.class, true));

- `TimeoutRetryPolicy`: instead of counting attempts, this policy uses a clock. It will keep retrying as long as the total elapsed time is under the threshold.

In [ ]:
TimeoutRetryPolicy policy = new TimeoutRetryPolicy();
policy.setTimeout(5000L); // Keep retrying for up to 5 seconds

- `ExceptionClassifierRetryPolicy`: if Exception A occurs, use `SimpleRetryPolicy(5)`. If Exception B occurs, use `NeverRetryPolicy`.

In [ ]:
ExceptionClassifierRetryPolicy policy = new ExceptionClassifierRetryPolicy();
policy.setPolicyMap(Map.of(
    RemoteAccessException.class, new SimpleRetryPolicy(5), // Retry 5 times for Network
    NullPointerException.class, new NeverRetryPolicy()     // Never retry code bugs
));

- `CompositeRetryPolicy`: ombine multiple policies using a consensus. For example, retry until either 10 attempts are reached OR 30 seconds have passed.

### Backoff Policy
`BackoffPoliy` determines the delay between consecutive retry attempts. Some built-in implementations:
- `FixedBackOffPolicy`: default policy, pauses for a constant, static period after every failure.

In [ ]:
// Equivalent to 
@Retryable(backoff = @Backoff(delay = 2000))

- `ExponentialBackOffPolicy`: increases the wait time after each failure, which is essential for giving a struggling downstream system time to recover

In [ ]:
// Equivalent to
@Retryable(backoff = @Backoff(delay = 1000, multiplier = 2.0, maxDelay = 10000))

- `ExponentialRandomBackOffPolicy`: introduces random jitter

In [ ]:
// Equivalent to
@Retryable(backoff = @Backoff(delay = 1000, multiplier = 2.0, maxDelay = 10000, random = true))

- `UniformRandomBackOffPolicy`: picks a random value from a fixed range for every retry. Unlike exponential backoff, the range does not grow.

In [ ]:
// Equivalent to
@Retryable(backoff = @Backoff(delay = 1000, maxDelay = 5000, random = true))

- `NoBackoffPolicy`: immediate retry.